<a href="https://colab.research.google.com/github/akash166d/sparkNLP/blob/master/NLP_classifier_ex_SparkNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Package/Lib

This is required to ensure the ubuntu related dependencies are up to date

In [ ]:
! sudo apt-get update --fix-missing
! sudo apt-get upgrade --fix-missing

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [255 kB]
Get:12 http://

In [ ]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.5.4

! pip install --user -U nltk

openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
     |████████████████████████████████| 215.7MB 55kB/s 
     |████████████████████████████████| 204kB 37.1MB/s 
     |████████████████████████████████| 133kB 5.4MB/s 
     |████████████████████████████████| 1.4MB 4.4MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434673 sha256=f05d72723947b9da775716adbbfb7d1e24d90b0831abdbf710ab2cc30c9ac703
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version; ", spark.version)

Spark NLP version:  2.5.4
Apache Spark version;  2.4.4


# Mount Drive and read data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


Some of the sparkNLP annotators were having trouble reading spaces in path, hence used symbolic link to replace the path

In [ ]:
! ln -s "/content/drive/My Drive" "/content/MyDrive"

In [ ]:

song_data = spark.read.format("csv").option("header", "true").load("/content/drive/My Drive/SparkNLP/song_2k.csv")


In [ ]:
from pyspark.sql import functions as F
song_data = song_data.select(['Key', 'Lyric', 'Genre']).withColumnRenamed('Lyric','text')

# train, test = trainDataset.randomSplit(weights=[0.5, 0.5], seed=123)
song_data = song_data.limit(999)
print(song_data.count())

song_data = song_data.filter(song_data.text != '')
print(song_data.count())
song_data = song_data.filter((song_data.Genre == 'Rock') | (song_data.Genre == 'Hip Hop') | (song_data.Genre == 'Pop')  )
# song_data = song_data.filter(song_data.Genre != '')
print(song_data.count())
# song_data = song_data.filter(song_data.text != ' ')
# print(song_data.count())
# song_data = song_data.filter(song_data.Genre != ' ')
# print(song_data.count())
song_data.na.drop(subset=["text"])
print(song_data.count())
song_data.show()

999
999
915
915
+--------------------+--------------------+-------+
|                 Key|                text|  Genre|
+--------------------+--------------------+-------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|
|jamiroquai_Rock D...|And it's coming a...|    Pop|
|10000 maniacs_The...|These are. These ...|   Rock|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|
|black veil brides...|Have we begun to ...|   Rock|
|lynyrd skynyrd_I ...|Ain't no need to ...|   Rock|
|10000 maniacs_Acr...|Well they left th...|   Rock|
|10000 maniacs_Pla...|[ music: Dennis D...|   Rock|
|10000 maniacs_Rai...|On bended kneeI'v...|   Rock|
|twista_Back 2 School|[Tung Twista]. ba...|Hip Hop|
|10000 maniacs_Ant...|For whom do the b...|   Rock|
|10000 maniacs_All...|She walks alone o...|   Rock|
|10000 maniacs_Bac...|Jenny. Jenny you ...|   Rock|
|cyndi lauper_True...|You with the sad ...|    P

In [ ]:
print(type(song_data))

<class 'pyspark.sql.dataframe.DataFrame'>


# Package Import

In [ ]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from pyspark.sql import functions as F

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Vocab Reduction

Basic Data cleaning for NLP

In [ ]:
assembler = DocumentAssembler().setInputCol('text').setOutputCol('document')

sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')#.setTargetPattern('/\b(\?You were looking)\b/')#.setExceptionsPath('/content/MyDrive/SparkNLP/entities.txt')

spell_chk = NorvigSweetingModel().pretrained().setInputCols(['token']).setOutputCol('corrected')

lemmatizer = LemmatizerModel().pretrained().setInputCols(['corrected']).setOutputCol('lemma')

normalizer = Normalizer().setInputCols(['lemma']).setOutputCol('normalized').setLowercase(True)

stop_wrd = list( stopwords.words('english'))

stop_words_cleaner = StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False).setStopWords(stop_wrd)

finisher = Finisher().setInputCols(['cleanTokens']).setOutputCols(['cleanTokens']).setOutputAsArray(True)

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [ ]:
pipeline_bow = Pipeline().setStages([
    assembler, tokenizer, spell_chk , 
    lemmatizer, normalizer,stop_words_cleaner,finisher
])
model_trans_tfIdf = pipeline_bow.fit(song_data)
model_trans_tfIdf =  model_trans_tfIdf.transform(song_data)
model_trans_tfIdf.show()

+--------------------+--------------------+-------+--------------------+
|                 Key|                text|  Genre|         cleanTokens|
+--------------------+--------------------+-------+--------------------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[could, feel, tim...|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[take, baby, hold...|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[come, baby, yeah...|
|10000 maniacs_The...|These are. These ...|   Rock|[day, youll, reme...|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|[grudge, slowly, ...|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|[donut, talk, lis...|
|black veil brides...|Have we begun to ...|   Rock|[begin, drift, aw...|
|lynyrd skynyrd_I ...|Ain't no need to ...|   Rock|[aint, need, worr...|
|10000 maniacs_Acr...|Well they left th...|   Rock|[well, leave, mor...|
|10000 maniacs_Pla...|[ music: Dennis D...|   Rock|[music, dennis, d...|
|10000 maniacs_Rai...|On bended kneeI'v...|   Rock|

# Vectorization

## BoW Method

In [ ]:
from pyspark.ml.feature import CountVectorizer, IDF

count_vectorizer = CountVectorizer(inputCol='cleanTokens', outputCol='tf', minDF=10)
cvmodel = count_vectorizer.fit(model_trans_tfIdf)
idf = IDF(inputCol='tf', outputCol='features', minDocFreq=10)
# finisher_idf = Finisher().setInputCols(['features']).setOutputCols(['tfidf']) finisher is for sparkNLP not spark ML
bow_pipeline = Pipeline().setStages([count_vectorizer, idf])
bow_pipeline = bow_pipeline.fit(model_trans_tfIdf)

bows = bow_pipeline.transform(model_trans_tfIdf)
bows.show()

+--------------------+--------------------+-------+--------------------+--------------------+--------------------+
|                 Key|                text|  Genre|         cleanTokens|                  tf|            features|
+--------------------+--------------------+-------+--------------------+--------------------+--------------------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[could, feel, tim...|(881,[0,1,3,4,8,1...|(881,[0,1,3,4,8,1...|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[take, baby, hold...|(881,[0,6,7,10,12...|(881,[0,6,7,10,12...|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[come, baby, yeah...|(881,[0,1,2,3,4,5...|(881,[0,1,2,3,4,5...|
|10000 maniacs_The...|These are. These ...|   Rock|[day, youll, reme...|(881,[1,9,11,15,1...|(881,[1,9,11,15,1...|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|[grudge, slowly, ...|(881,[4,12,24,80,...|(881,[4,12,24,80,...|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|[donut, talk, lis...|(881,[0,

## Approach 1: Top 3 > Embeddings

Vectorization create vector column object and we need to retrieve values and indices accordingly from it unlike python array/list
This whole para shows how we can retrieve indices and ranking from feature column in various ways by applying UDF

In [ ]:
from collections import Counter
from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *
# retrieve column with dict indices>TFIDF_Rank 
# @udf("map<long, double>")
# def vector_as_map(v):
#    if isinstance(v, SparseVector):
#        return dict(zip(v.indices.tolist(), v.values.tolist()))
       
#    elif isinstance(v, DenseVector):
#       print("x")
#       return dict(zip(range(len(v)), v.values.tolist()))

# bows_wrdlevel = bows.select("cleanTokens", vector_as_map("features").alias("rating"))
# bows_wrdlevel.show(truncate = False)

# retrieve Columns with dict of only prominent word (based on rating)
 
# schema = ArrayType(StructType([
#     StructField("ind", IntegerType(), False),
#     StructField("rat", DoubleType(), False)
# ]))

# @udf(schema)
# def vector_as_map(v):
#    if isinstance(v, SparseVector):
#       K = Counter(dict(zip(v.indices.tolist(), v.values.tolist())))
#       return K.most_common(3)
#    elif isinstance(v, DenseVector):
#       L =Counter(dict(zip(range(len(v)), v.values.tolist())))
#       return L.most_common(3) 

# bows_wrdlevel = bows.select("cleanTokens", vector_as_map("features").alias("rating"))
# bows_wrdlevel.show(truncate = False)

# getting top 3 indices without retaining their rank

# schema = ArrayType(StructType([
#     StructField("ind", IntegerType(), False),
#     StructField("rat", DoubleType(), False)
# ]))

@udf(ArrayType(IntegerType()))
def vector_as_map(v):
   if isinstance(v, SparseVector):
      K = dict(zip(v.indices.tolist(), v.values.tolist()))
      top3 = sorted(K, key=K.get, reverse=True)[:3]
      return top3
   elif isinstance(v, DenseVector):
      L =dict(zip(range(len(v)), v.values.tolist()))
      top3 = sorted(L, key=L.get, reverse=True)[:3]
      return top3 

bows_wrdlevel = bows.select("Key",'Genre', vector_as_map("features").alias("top_ind"))
bows_wrdlevel.na.drop(subset=["top_ind"])
bows_wrdlevel.show(truncate = True)

+--------------------+-------+---------------+
|                 Key|  Genre|        top_ind|
+--------------------+-------+---------------+
|10000 maniacs_Mor...|   Rock|  [70, 57, 718]|
|10000 maniacs_Bec...|   Rock| [320, 54, 424]|
|jamiroquai_Rock D...|    Pop|[609, 588, 249]|
|10000 maniacs_The...|   Rock| [61, 173, 555]|
|10000 maniacs_Eve...|   Rock|[413, 768, 682]|
|10000 maniacs_Don...|   Rock| [107, 43, 828]|
|black veil brides...|   Rock|  [6, 147, 337]|
|lynyrd skynyrd_I ...|   Rock|    [41, 6, 33]|
|10000 maniacs_Acr...|   Rock|[298, 490, 226]|
|10000 maniacs_Pla...|   Rock|[832, 195, 716]|
|10000 maniacs_Rai...|   Rock| [660, 147, 53]|
|twista_Back 2 School|Hip Hop|[378, 188, 249]|
|10000 maniacs_Ant...|   Rock|[761, 227, 744]|
|10000 maniacs_All...|   Rock| [155, 65, 461]|
|10000 maniacs_Bac...|   Rock|[353, 692, 370]|
|cyndi lauper_True...|    Pop|[445, 173, 223]|
|10000 maniacs_A R...|   Rock| [63, 251, 807]|
|rick astley_She M...|    Pop| [30, 156, 203]|
|steve earle_

Convert indices to word

In [ ]:
from pyspark.sql import functions as F
from operator import itemgetter

vocab = cvmodel.vocabulary
# udf_to_words = udf(get_words, ArrayType(StringType()))
# use @udf if you have used that pattern, throughout your pynb
@udf(ArrayType(StringType()))
def get_words(token_in):
    return itemgetter(*token_in)(vocab)
    # return map(vocab.__getitem__, token_in)
    # return vocab[token_in].tolist()


This gives us top 3 word for each doc

In [ ]:
bow_top3_wrd = bows_wrdlevel.select('Key','Genre',get_words(F.col('top_ind')).alias('word'))
bow_top3_wrd.na.drop(subset=["word"])
bow_top3_wrd.show()
# bow_top5_wrd.schema


+--------------------+-------+--------------------+
|                 Key|  Genre|                word|
+--------------------+-------+--------------------+
|10000 maniacs_Mor...|   Rock|[theres, nothing,...|
|10000 maniacs_Bec...|   Rock|[belong, night, l...|
|jamiroquai_Rock D...|    Pop|[coming, page, rock]|
|10000 maniacs_The...|   Rock|[youll, true, bless]|
|10000 maniacs_Eve...|   Rock|[everyday, pack, ...|
|10000 maniacs_Don...|   Rock|[really, keep, th...|
|black veil brides...|   Rock| [oh, save, mistake]|
|lynyrd skynyrd_I ...|   Rock|    [baby, oh, need]|
|10000 maniacs_Acr...|   Rock|[anymore, trouble...|
|10000 maniacs_Pla...|   Rock|[brain, truth, st...|
|10000 maniacs_Rai...|   Rock|[theyve, save, wo...|
|twista_Back 2 School|Hip Hop| [school, use, rock]|
|10000 maniacs_Ant...|   Rock|[heat, fire, surr...|
|10000 maniacs_All...|   Rock|[forever, hold, s...|
|10000 maniacs_Bac...|   Rock|[angel, style, shes]|
|cyndi lauper_True...|    Pop|[mama, true, afraid]|
|10000 mania

### Embedding

embedding works on sentence as it tries to find out the relationship between words too and not just their count/occurence. Because of this we have Approach 2 where we are creating embedding first and then doing picking embeddings for top 3 words.

In [ ]:
#convert list to sentence
@udf( StringType())
def get_sentence(word_in):
    return " ".join(str(item) for item in word_in)

# udf_to_doc = udf(get_sentence, StringType())

bow_top_embd = bow_top3_wrd.select('Key','Genre', get_sentence(F.col('word')).alias('doc'))
# bow_top_embd = bow_top_embd.filter((F.col('doc') != '') | F.col('doc') != ' ')
bow_top_embd.na.drop(subset=["doc"])
from pyspark.sql.functions import trim
bow_top_embd = bow_top_embd.withColumn("doc", trim(bow_top_embd.doc))
# bow_top_embd.filter(!(F.col('doc') == ' ')).show()
bow_top_embd.show()

+--------------------+-------+--------------------+
|                 Key|  Genre|                 doc|
+--------------------+-------+--------------------+
|10000 maniacs_Mor...|   Rock|nothing theres shout|
|10000 maniacs_Bec...|   Rock|  belong night lover|
|jamiroquai_Rock D...|    Pop|    coming page rock|
|10000 maniacs_The...|   Rock|    youll true bless|
|10000 maniacs_Eve...|   Rock|everyday pack silent|
|10000 maniacs_Don...|   Rock|   really keep three|
|black veil brides...|   Rock|     oh save mistake|
|lynyrd skynyrd_I ...|   Rock|        baby oh need|
|10000 maniacs_Acr...|   Rock|anymore trouble part|
|10000 maniacs_Pla...|   Rock|brain truth strength|
|10000 maniacs_Rai...|   Rock|   theyve save would|
|twista_Back 2 School|Hip Hop|     school use rock|
|10000 maniacs_Ant...|   Rock|  heat fire surround|
|10000 maniacs_All...|   Rock|  forever hold sorry|
|10000 maniacs_Bac...|   Rock|    angel style shes|
|cyndi lauper_True...|    Pop|    mama true afraid|
|10000 mania

In [ ]:
bow_top_embd.printSchema()
bow_top_embd.select('Genre').distinct().collect()

root
 |-- Key: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- doc: string (nullable = true)



[Row(Genre='Rock'), Row(Genre='Pop'), Row(Genre='Hip Hop')]

Embedding output here will be list of list 3*100 features

In [ ]:
assembler = DocumentAssembler().setInputCol('doc').setOutputCol("document").setCleanupMode("shrink")
tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')
embed = WordEmbeddingsModel.pretrained().setInputCols(['document',"token"]).setOutputCol("embeddings")
embeddings_finisher = EmbeddingsFinisher().setInputCols(["embeddings"]).setOutputCols(["embeddings"]).setOutputAsVector(True).setCleanAnnotations(False)
finisher = Finisher().setInputCols(['token']).setOutputCols(['token']).setOutputAsArray(True)
# vec_assembler = VectorAssembler().setInputCols(['embeddings']).setOutputCol('features')
pipeline_uni = Pipeline().setStages([
    assembler, tokenizer,embed ,embeddings_finisher, finisher
])
model = pipeline_uni.fit(bow_top_embd)
model_trans =  model.transform(bow_top_embd)#.persist()
# model_trans.select('embeddings').show(truncate = False)
model_trans.show(10)


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
+--------------------+-----+--------------------+--------------------+--------------------+
|                 Key|Genre|                 doc|               token|          embeddings|
+--------------------+-----+--------------------+--------------------+--------------------+
|10000 maniacs_Mor...| Rock|nothing theres shout|[nothing, theres,...|[[0.1090399995446...|
|10000 maniacs_Bec...| Rock|  belong night lover|[belong, night, l...|[[0.0577719993889...|
|jamiroquai_Rock D...|  Pop|    coming page rock|[coming, page, rock]|[[0.0667089968919...|
|10000 maniacs_The...| Rock|    youll true bless|[youll, true, bless]|[[-0.556039988994...|
|10000 maniacs_Eve...| Rock|everyday pack silent|[everyday, pack, ...|[[-0.350219994783...|
|10000 maniacs_Don...| Rock|   really keep three|[really, keep, th...|[[-0.054349001497...|
|black veil brides...| Rock|     oh save mistake| [oh, save, mistake]|[[-0.

String Indexer to change string categories to label for classification

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "Genre", outputCol = "label" ,)
pipeline = Pipeline(stages=[ label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(model_trans)
dataset = pipelineFit.transform(model_trans)
selectedCols = ['token','embeddings','label']
dataset = dataset.select(selectedCols)
dataset.show(5)
# dataset.printSchema()


+--------------------+--------------------+-----+
|               token|          embeddings|label|
+--------------------+--------------------+-----+
|[theres, nothing,...|[[-0.123620003461...|  0.0|
|[belong, night, l...|[[0.0577719993889...|  0.0|
|[coming, page, rock]|[[0.0667089968919...|  1.0|
|[youll, true, bless]|[[-0.556039988994...|  0.0|
|[everyday, pack, ...|[[-0.350219994783...|  0.0|
+--------------------+--------------------+-----+
only showing top 5 rows



In [ ]:
dataset.select('label').distinct().collect()

[Row(label=0.0), Row(label=1.0), Row(label=2.0)]

Spark ML takes vector as input so need to convert our embedding output to vector. We are going to use dense vector here. 

In [ ]:
import itertools
import functools
from pyspark.ml.linalg import Vectors, VectorUDT
@udf(VectorUDT())
# @udf(StringType())
def get_Flatvector(embd_vec):
  # embd_vec = list(embd_vec)
  merged = list(itertools.chain(*embd_vec))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  mer_floats = [float(np_float) for np_float in merged]
  ver = Vectors.dense(mer_floats)
  return ver
  # return str(type(mer_floats[0]))
  # return embd_vec[0].tolist()

dataset_clsf = dataset.select('token',get_Flatvector(F.col('embeddings')).alias('features'),'label')
dataset_clsf.na.drop(subset=["features"])
# dataset_clsf = dataset_clsf.where("'label' != ''").where("'label' != ' '")
# dataset_clsf = dataset_clsf.filter(dataset_clsf.label == 0.0)
dataset_clsf.show(truncate = True)

# dataset.select("label",flatten("embeddings")).show(false)

+--------------------+--------------------+-----+
|               token|            features|label|
+--------------------+--------------------+-----+
|[theres, nothing,...|[-0.1236200034618...|  0.0|
|[belong, night, l...|[0.05777199938893...|  0.0|
|[coming, page, rock]|[0.06670899689197...|  1.0|
|[youll, true, bless]|[-0.5560399889945...|  0.0|
|[everyday, pack, ...|[-0.3502199947834...|  0.0|
|[really, keep, th...|[-0.0543490014970...|  0.0|
| [oh, save, mistake]|[-0.6666499972343...|  0.0|
|    [baby, oh, need]|[0.49746999144554...|  0.0|
|[anymore, trouble...|[0.38657999038696...|  0.0|
|[brain, truth, st...|[-0.1024399995803...|  0.0|
|[theyve, save, wo...|[0.0,0.0,0.0,0.0,...|  0.0|
| [school, use, rock]|[0.80335998535156...|  2.0|
|[heat, fire, surr...|[-0.8194400072097...|  0.0|
|[forever, hold, s...|[-0.0296199992299...|  0.0|
|[angel, style, shes]|[0.34995999932289...|  0.0|
|[mama, true, afraid]|[0.27950999140739...|  1.0|
|   [wait, room, dry]|[-0.1498499959707...|  0.0|


We need to ensure that the size of feature is 300. group by count of feature to check and remove unwanted rows. This is probably because TFIDF gave same rank to multiple words and we have more than 3 in some cases. in our case it was just one row

In [ ]:
from pyspark.sql import functions as F
import itertools
import functools
from pyspark.ml.linalg import Vectors, VectorUDT
# @udf(VectorUDT())
@udf(IntegerType())
def get_Flatvector_len(embd_vec):
  # embd_vec = list(embd_vec)
  # merged = list(itertools.chain(*embd_vec))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  # mer_floats = [float(np_float) for np_float in merged]
  vec =Vectors.dense(embd_vec)
  return (len(vec))
  # return str(type(mer_floats[0]))
  # return embd_vec[0].tolist()

dataset_clsf_300 = dataset_clsf.select('token','features',get_Flatvector_len(F.col('features')).alias('feat_cnt'),'label')
dataset_clsf_300.na.drop(subset=["feat_cnt"])
dataset_clsf_300.na.drop(subset=["features"])
dataset_clsf_300.na.drop(subset=["label"])
# dataset_clsf.show(truncate = True)
# dataset_clsf = dataset_clsf.groupBy("features").agg(F.count('features'))
dataset_clsf_300 = dataset_clsf_300.filter((dataset_clsf_300.feat_cnt == 300) | (dataset_clsf_300.label != ' ') ).select('token','features','label')
# dataset_clsf_300 = dataset_clsf_300.filter(dataset_clsf_300.label != ' ').select('features','label')
# dataset_clsf_300 = dataset_clsf_300.filter(F.col('features') != ' ')
# dataset_clsf_300.show(dataset_clsf_300.count() )
# dataset_clsf_300.repartition(1).count()
dataset_clsf_300.show()
# dataset_clsf_300.count()
# dataset_clsf_300.groupBy("feat_cnt").agg(F.count('feat_cnt')).show()

+--------------------+--------------------+-----+
|               token|            features|label|
+--------------------+--------------------+-----+
|[theres, nothing,...|[-0.1236200034618...|  0.0|
|[belong, night, l...|[0.05777199938893...|  0.0|
|[coming, page, rock]|[0.06670899689197...|  1.0|
|[youll, true, bless]|[-0.5560399889945...|  0.0|
|[everyday, pack, ...|[-0.3502199947834...|  0.0|
|[really, keep, th...|[-0.0543490014970...|  0.0|
| [oh, save, mistake]|[-0.6666499972343...|  0.0|
|    [baby, oh, need]|[0.49746999144554...|  0.0|
|[anymore, trouble...|[0.38657999038696...|  0.0|
|[brain, truth, st...|[-0.1024399995803...|  0.0|
|[theyve, save, wo...|[0.0,0.0,0.0,0.0,...|  0.0|
| [school, use, rock]|[0.80335998535156...|  2.0|
|[heat, fire, surr...|[-0.8194400072097...|  0.0|
|[forever, hold, s...|[-0.0296199992299...|  0.0|
|[angel, style, shes]|[0.34995999932289...|  0.0|
|[mama, true, afraid]|[0.27950999140739...|  1.0|
|   [wait, room, dry]|[-0.1498499959707...|  0.0|


In [ ]:
# set seed for reproducibility
# train, test = dataset_clsf_300.randomSplit(weights=[0.6, 0.4], seed=100)
# dataset_clsf_300 = dataset_clsf_300.where("'features' != ''")
(trainingData, testData) = dataset_clsf_300.select('token','features','label').randomSplit([0.7, 0.3], seed = 100)
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))
# train, test = dataset_clsf_300.randomSplit([0.6, 0.4], seed=100)
trainingData.show(10)

+--------------------+--------------------+-----+
|               token|            features|label|
+--------------------+--------------------+-----+
|      [act, eye, uh]|[0.24027000367641...|  0.0|
|[afraid, perfect,...|[0.20393000543117...|  0.0|
|[afraid, perfect,...|[0.20393000543117...|  0.0|
|[afraid, sweet, p...|[0.20393000543117...|  0.0|
|   [ah, baby, steal]|[-1.4133000373840...|  1.0|
|[ah, believe, fri...|[-1.4133000373840...|  0.0|
| [alone, goin, help]|[0.38927999138832...|  0.0|
|[alone, wrong, ni...|[0.38927999138832...|  0.0|
|[already, round, ...|[0.13563999533653...|  0.0|
|[already, round, ...|[0.13563999533653...|  0.0|
+--------------------+--------------------+-----+
only showing top 10 rows



### Classification

### Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label',maxIter=20, regParam=0.3, elasticNetParam=0   )
lrModel = lr.fit(trainingData )
predictions = lrModel.transform(testData)
predictions.show()

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|               token|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
| [ago, rock, become]|[0.54666000604629...|  0.0|[1.41761643875231...|[0.79741206092989...|       0.0|
|   [ah, yeah, crazy]|[-1.4133000373840...|  1.0|[-0.1186564800670...|[0.24700552033410...|       1.0|
|[aint, call, always]|[-0.1444900035858...|  1.0|[1.66194859571792...|[0.83348033887642...|       0.0|
|[alive, night, ever]|[-0.0055056000128...|  1.0|[1.50112892351291...|[0.80985501917549...|       0.0|
|[already, young, ...|[0.13563999533653...|  0.0|[1.28606549779604...|[0.75176838242710...|       0.0|
|  [alright, oh, try]|[0.34696000814437...|  1.0|[1.30015038665708...|[0.77115126204828...|       0.0|
|[angel, style, shes]|[0.34995999932289...|  0.0|[1.67267610797735...|[0.

In [ ]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)
# predictions.show()

0.735822417115046

#### Top Words

In [ ]:
top_wrd_df = predictions.filter(predictions.label == predictions.prediction)
top_wrd_df.show()
top_wrd_df.count()

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|               token|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
| [ago, rock, become]|[0.54666000604629...|  0.0|[1.41761643875231...|[0.79741206092989...|       0.0|
|   [ah, yeah, crazy]|[-1.4133000373840...|  1.0|[-0.1186564800670...|[0.24700552033410...|       1.0|
|[already, young, ...|[0.13563999533653...|  0.0|[1.28606549779604...|[0.75176838242710...|       0.0|
|[angel, style, shes]|[0.34995999932289...|  0.0|[1.67267610797735...|[0.82726193161393...|       0.0|
| [angel, today, gun]|[0.34995999932289...|  0.0|[1.45533061587012...|[0.78274042762116...|       0.0|
|[anymore, trouble...|[0.38657999038696...|  0.0|[1.94382655069015...|[0.85622207318998...|       0.0|
|[anymore, trouble...|[0.38657999038696...|  0.0|[1.94382655069015...|[0.

220

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(top_wrd_df['label']).orderBy(top_wrd_df['probability'].desc())

top_wrd_df = top_wrd_df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 10)
top_wrd_df.count()

21

In [ ]:
top_wrd_df_lst = top_wrd_df.groupBy('label').agg(F.collect_list("token"))
top_wrd_df_lst.show(truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|collect_list(token)                                                                                                                                                                                                          |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |[[human, mistake, watch], [put, fire, set], [spit, regret, round], [screen, yesterday, young], [stay, lot, till], [inside, deep, head], [fix, plain, thin], [black, fate, end], [circle, line, stone], [mirror, clear, away]]|
|1.0  |[[oh, wasnt, cry], [love, hello, feelings], [oh, end, girl], [oh, care, w

In [ ]:
# trainingSummary = lrModel.summary
# roc = trainingSummary.roc.toPandas()
# plt.plot(roc['FPR'],roc['TPR'])
# plt.ylabel('False Positive Rate')
# plt.xlabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.show()
# print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

### Naive Bayes

We can only use Naive Bayes for positive values, we have negative too.

In [ ]:
# from pyspark.ml.classification import NaiveBayes
# nb = NaiveBayes(smoothing=1)
# model = nb.fit(trainingData)
# predictions = model.transform(testData)
# # predictions.filter(predictions['prediction'] == 0) \
# #     .select("Descript","Category","probability","label","prediction") \
# #     .orderBy("probability", ascending=False) \
# #     .show(n = 10, truncate = 30)
# predictions.show()

### Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
algo = RandomForestClassifier(featuresCol='features', labelCol='label')
model = algo.fit(trainingData)
predictions = model.transform(testData)
predictions.show()


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
accracy = evaluator.evaluate(predictions)
accracy

In [ ]:
# #important: need to cast to float type, and order by prediction, else it won't work
# preds_and_labels = predictions.select(['prediction','d']).withColumn('label', F.col('d').cast(FloatType())).orderBy('prediction')

# #select only prediction and label columns
# preds_and_labels = preds_and_labels.select(['prediction','label'])

# metrics = MultiClassMetrics(preds_and_labels.rdd.map(tuple))

y_true = predictions.select(['label']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

## Approach 2: Embeddings > Top 3

Start with creating embeddings first

In [ ]:
assembler = DocumentAssembler().setInputCol('text').setOutputCol('document')

sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')#.setTargetPattern('/\b(\?You were looking)\b/')#.setExceptionsPath('/content/MyDrive/SparkNLP/entities.txt')

spell_chk = NorvigSweetingModel().pretrained().setInputCols(['token']).setOutputCol('corrected')

lemmatizer = LemmatizerModel().pretrained().setInputCols(['corrected']).setOutputCol('lemma')

normalizer = Normalizer().setInputCols(['lemma']).setOutputCol('normalized').setLowercase(True)

stop_wrd = list( stopwords.words('english'))

stop_words_cleaner = StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False).setStopWords(stop_wrd)

embed = WordEmbeddingsModel.pretrained().setInputCols(['document',"token"]).setOutputCol("embeddings")
embeddings_finisher = EmbeddingsFinisher().setInputCols(["embeddings"]).setOutputCols(["embeddings"]).setOutputAsVector(True).setCleanAnnotations(False)

finisher = Finisher().setInputCols(['cleanTokens']).setOutputCols(['cleanTokens']).setOutputAsArray(True).setCleanAnnotations(True)

pipeline_embd = Pipeline().setStages([ assembler, tokenizer, spell_chk , lemmatizer, normalizer,stop_words_cleaner,embed,embeddings_finisher,finisher ]) 
model_trans_tfIdf_2 = pipeline_embd.fit(song_data) 
model_trans_tfIdf_2 = model_trans_tfIdf_2.transform(song_data) 
model_trans_tfIdf_2.show(10)
# model_trans_tfIdf_2.select('embeddings').show(10,truncate = False)

1. Create TFIDF
2. pick top 3 position where rank is max
3. pick embeddings for that top 3 position
4. map it to original words

In [ ]:
from pyspark.ml.feature import CountVectorizer, IDF

count_vectorizer = CountVectorizer(inputCol='cleanTokens', outputCol='tf', minDF=10)
cvmodel = count_vectorizer.fit(model_trans_tfIdf_2)
idf = IDF(inputCol='tf', outputCol='features', minDocFreq=10)
# finisher_idf = Finisher().setInputCols(['features']).setOutputCols(['tfidf']) finisher is for sparkNLP not spark ML
bow_pipeline_2 = Pipeline().setStages([count_vectorizer, idf])
bow_pipeline_2 = bow_pipeline_2.fit(model_trans_tfIdf_2)

bows_2 = bow_pipeline_2.transform(model_trans_tfIdf_2)
bows_2.show()

In [ ]:
from collections import Counter
from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *

@udf(ArrayType(IntegerType()))
def get_pos_top3(v):
  #  if isinstance(v, SparseVector):
    rank = v.values.tolist()
    top_3_pos =sorted( sorted(range(len(rank)), key=lambda i: rank[i])[-3:])
      # K = dict(zip(v.indices.tolist(), v.values.tolist()))
      # top3 = sorted(K, key=K.get, reverse=True)[:3]
    return top_3_pos
  #  elif isinstance(v, DenseVector):
  #     L =dict(zip(range(len(v)), v.values.tolist()))
  #     top3 = sorted(L, key=L.get, reverse=True)[:3]
  #     return top3 

bows_wrdlevel_2 = bows_2.select("Key",'Genre', get_pos_top3("features").alias("top3_pos"),'embeddings')
bows_wrdlevel_2.na.drop(subset=["top3_pos"])
bows_wrdlevel_2.show()
# bows_wrdlevel_2.select('embeddings').show(truncate = False)

Select embeddings at those top 3 positions

In [ ]:
from collections import Counter
from pyspark.sql.functions import udf, explode
from pyspark.sql.functions import struct, array
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors, VectorUDT

# @udf(ArrayType(ArrayType(FloatType())))
# @udf(ArrayType(IntegerType()))
# @udf(StringType())
# @udf(IntegerType())
@udf(VectorUDT())
def get_embd_top3(v):
  
  # merged = list(itertools.chain(*embedd))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  # mer_floats = [float(np_float) for np_float in merged]
  # ver = Vectors.dense(mer_floats)
  #create list of list from embedding using positions
  indices = v[0]
  embedd = v[1]
  embd_top3 = []
  
  for i in indices:
    sublist = [float(np_float) for np_float in embedd[i]]
    embd_top3.append(sublist)
  
  #flatten out the list
  merged = list(itertools.chain(*embd_top3))
  
  #convert to vector and send
  ver = Vectors.dense(merged)

  # tp = type(merged[0])
  return ver
  # return str(tp)
  # return len(merged) 

bows_embed_2 = bows_wrdlevel_2.select("Key",'Genre', get_embd_top3(struct('top3_pos', 'embeddings')).alias("features"))
bows_embed_2.na.drop(subset=["features"])
bows_embed_2.show(truncate = False)
# bows_wrdlevel_2.select('embeddings').show(truncate = False)

1. filter out column != 300
2. stringlabel indexer

In [ ]:
from pyspark.sql import functions as F
import itertools
import functools
from pyspark.ml.linalg import Vectors, VectorUDT
# @udf(VectorUDT())
@udf(IntegerType())
def get_Flatvector_len_2(embd_vec):
  # embd_vec = list(embd_vec)
  # merged = list(itertools.chain(*embd_vec))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  # mer_floats = [float(np_float) for np_float in merged]
  vec =Vectors.dense(embd_vec)
  return (len(vec))
  # return str(type(mer_floats[0]))
  # return embd_vec[0].tolist()

dataset_clsf_300_2 = bows_embed_2.select('features',get_Flatvector_len_2(F.col('features')).alias('feat_cnt'),'Genre')
dataset_clsf_300_2.na.drop(subset=["feat_cnt"])
dataset_clsf_300_2.na.drop(subset=["features"])
# dataset_clsf_300.na.drop(subset=["label"])
# dataset_clsf.show(truncate = True)
# dataset_clsf = dataset_clsf.groupBy("features").agg(F.count('features'))
dataset_clsf_300_2 = dataset_clsf_300_2.filter((dataset_clsf_300_2.feat_cnt == 300) | (dataset_clsf_300_2.Genre != ' ') ).select('features','Genre')
# dataset_clsf_300 = dataset_clsf_300.filter(dataset_clsf_300.label != ' ').select('features','label')
# dataset_clsf_300 = dataset_clsf_300.filter(F.col('features') != ' ')
# dataset_clsf_300.show(dataset_clsf_300.count() )
# dataset_clsf_300.repartition(1).count()

dataset_clsf_300_2.show()
# dataset_clsf_300.groupBy("feat_cnt").agg(F.count('feat_cnt')).show()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "Genre", outputCol = "label" ,)
pipeline = Pipeline(stages=[ label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(dataset_clsf_300_2)
dataset = pipelineFit.transform(dataset_clsf_300_2)
selectedCols = ['features','label']
dataset = dataset.select(selectedCols)
dataset.show(5)
dataset.select('label').distinct().collect()

continue classification from here.............................